In [ ]:
import rubin_nights.dayobs_utils as rn_dayobs

day_obs = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
day_obs = 20251101
sim_nights = 1

In [ ]:
import os
if os.getenv("EXTERNAL_INSTANCE_URL") is not None:
    print("updating rubin_nights")
    !pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git  --no-deps #  > /dev/null 2>&1
    print("updating ts_fbs_utils")
    !pip install --user --upgrade git+https://github.com/lsst-ts/ts_fbs_utils.git --no-deps #> /dev/null 2>&1
    print("updating lsst_survey_sim")
    !pip install --user --upgrade git+https://github.com/lsst-sims/lsst_survey_sim.git --no-deps # > /dev/null 2&1

In [ ]:
import getpass
import sys

username = getpass.getuser()
location = os.getenv("EXTERNAL_INSTANCE_URL", "")

# RUBIN_SIM_DATA_DIR at usdf
if 'usdf' in os.getenv("EXTERNAL_INSTANCE_URL", ""):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# TOKEN CONFIGURATION
if location != "":
    # You are on an rsp.
    tokenfile = None
    site = None
# If you are outside of an RSP? - just use USDF and your own USDF-RSP token
# See https://rsp.lsst.io/guides/auth/creating-user-tokens.html
else:
    # Substitute the location of your own tokenfile
    # If you prefer, 'get_client' will also get token info from an "ACCESS_TOKEN" environment variable
    tokenfile = os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp")
    site = 'usdf'

# Fix for Eric's tokenfile location .. we should standardize this
if username=='neilsen' and location=="https://usdf-rsp.slac.stanford.edu":
    tokenfile = "/home/n/neilsen/.lsst/usdf_access_token"


# CONFIGURE SOME PATHS
ts_config_scheduler_root = None

if username=='neilsen' and location=="https://usdf-rsp.slac.stanford.edu":
    # Modifications of the python path with
    # sys.path do not work if the same namespace
    # is already scanned in the PYTHONPATH.
    # Get rid of the modules pre-loaded so the
    # interpreter reloads lsst.blah.blah.blah
    # after we have made our modifications
    # to sys.path.
    sys.modules.pop("lsst.ts.fbs", None)
    sys.modules.pop("lsst.ts", None)
    sys.modules.pop("lsst", None)
    
    sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.18.1")
    sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/SP-2167/rubin_sim")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_nights")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/lsst_survey_sim")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/ts_fbs_utils/python")

    ts_config_scheduler_root = "/sdf/data/rubin/user/neilsen/devel/ts_config_scheduler"
    do_git_stuff = False

elif username=='lynnej' and location == "":
    ts_config_scheduler_root = "/Users/lynnej/lsst_repos/ts_config_scheduler"
    do_git_stuff = True

# For everyone who did not set ts_config_scheduler_root path
if ts_config_scheduler_root is None:
    # Just make a new clone for ts_config_scheduler
    ts_config_scheduler_root = "."
    do_git_stuff = True


assert isinstance(ts_config_scheduler_root, str), "Please set ts_config_scheduler_root"

In [ ]:
import getpass
import os
import warnings
import copy
import logging

logging.getLogger("lsst_survey_sim").setLevel(logging.INFO)
logging.getLogger("rubin_nights").setLevel(logging.INFO)

import numpy as np
import pandas as pd
import sqlite3
import healpy as hp
import matplotlib.pyplot as plt
import colorcet as cc
import skyproj
from IPython.display import display, HTML

from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord
import astropy.units as u

from rubin_scheduler.scheduler.utils import SchemaConverter
from rubin_scheduler.site_models import Almanac
from rubin_scheduler.scheduler.features import Conditions
from rubin_scheduler.utils import ddf_locations, angular_separation, approx_ra_dec2_alt_az, Site, SURVEY_START_MJD

import rubin_sim.maf as maf
from rubin_sim.data import get_baseline

from rubin_nights import connections
import rubin_nights.lfa_data as rn_lfa
import rubin_nights.dayobs_utils as rn_dayobs
import rubin_nights.plot_utils as rn_plots
import rubin_nights.augment_visits as augment_visits
import rubin_nights.rubin_scheduler_addons as rn_sch
import rubin_nights.rubin_sim_addons as rn_sim
import rubin_nights.observatory_status as observatory_status
import rubin_nights.scriptqueue as scriptqueue
import rubin_nights.scriptqueue_formatting as scriptqueue_formatting

import importlib

from lsst_survey_sim import lsst_support, simulate_lsst, plot

band_colors = rn_plots.PlotStyles.band_colors

#%load_ext memory_profiler

In [ ]:
# Set this notebook to run today, for one night, by default.

# today_dayobs is the day of today - useful for checking if we're running in the past or not
today_dayobs = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())

# day_obs == the day to start the simulation
# Set in first cell

day_obs_time = rn_dayobs.day_obs_to_time(day_obs)
next_day_obs_time = rn_dayobs.day_obs_to_time(day_obs) + TimeDelta(1, format='jd')

next_day_obs = rn_dayobs.day_obs_str_to_int(rn_dayobs.time_to_day_obs(next_day_obs_time))

# Some parameters relating to downtime setup for model observatory
# In general, if only simulating one night, probably want no downtime or clouds
day_downtime = day_obs
if sim_nights == 1:
    add_downtime = False
    real_downtime = False
    add_clouds = False
else:
    add_downtime = True
    real_downtime = True
    add_clouds = True
# But could try to simulate previous nights to check if we recreate what happened 
if day_obs < today_dayobs:
    print("Checking the past, will restrict uptime to time acquiring science visits")
    add_downtime = True
    real_downtime = True
    add_clouds = False
    day_downtime = next_day_obs

real_downtime = False
add_clouds = True
print("Setting up to with downtimes like:")
print("add_downtime =", add_downtime)
print("real_downtime = ", real_downtime)
print("add_clouds = ", add_clouds)

survey_start = SURVEY_START_MJD
programs = ["BLOCK-365", "BLOCK-407", "BLOCK-408"]
print("Potential science programs : ", programs)


sunset, sunrise = rn_dayobs.day_obs_sunset_sunrise(day_obs, sun_alt=-12)
print(f"Simulation for {sim_nights} nights starting on :")
print(f"DayObs {day_obs}, -12 deg sunset {sunset.iso}, -12 deg sunrise {sunrise.iso}")

In [ ]:
endpoints = connections.get_clients(tokenfile=tokenfile, site=site)

In [ ]:
# run branch
q = endpoints['obsenv'].select_top_n("lsst.obsenv.run_branch", ['branch_name'], 1, time_cut=sunset)
q2 = endpoints['obsenv'].select_time_series("lsst.obsenv.run_branch", ["branch_name"], sunset, sunrise)
runbranch = pd.concat([q, q2])
display(runbranch)
current_runbranch = runbranch.branch_name.iloc[-1]
print(f"Current run branch {current_runbranch}")
# configuration information
q = endpoints['efd'].select_top_n('lsst.sal.Scheduler.logevent_configurationApplied', ["configurations", "salIndex", "private_origin", "url", "version"], 1, time_cut=sunset, index=1)
q2 = endpoints['efd'].select_time_series('lsst.sal.Scheduler.logevent_configurationApplied', ["configurations", "salIndex", "private_origin", "url", "version"], sunset, sunrise, index=1)
q = pd.concat([q, q2])
def strip_repo(x: pd.Series):
    return x.url.split("/")[-3]
def strip_version(x: pd.Series):
    return x.version.replace("heads/", "")
def strip_yaml(x: pd.Series):
    return x.configurations.split(",")[-1]
config_repo = q.apply(strip_repo, axis=1)
config_version = q.apply(strip_version, axis=1)
config_yaml = q.apply(strip_yaml, axis=1)
configs = pd.DataFrame([config_repo, config_version, config_yaml], columns=q.index, index=['config_repo', 'config_commit', 'config_yaml']).T
display(configs)

In [ ]:
# Git checkout ts_config_scheduler and set the configs to use.
# .. have to sort something out about what happens if the repo isn't clean ..
do_git_stuff = True
if do_git_stuff:
    ts_commit = 'develop'
    #ts_commit = current_runbranch
    simulate_lsst.get_configuration(ts_commit, clone_path=ts_config_scheduler_root)
config_script_path = os.path.join(ts_config_scheduler_root, "Scheduler/feature_scheduler/maintel/", "fbs_config_lsst_survey.py")
config_ddf_script_path = os.path.join(ts_config_scheduler_root, "Scheduler/ddf_gen/lsst_ddf_gen_block_407.py")

In [ ]:
# Retrieve previous visits to start scheduler 

refresh_visits = True

if refresh_visits:
    initial_opsim = simulate_lsst.fetch_previous_visits(next_day_obs, tokenfile=tokenfile, site=site)
    if initial_opsim is not None:
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always") 
            initial_opsim.to_hdf("opsim.h5", key='visits')
else:
    initial_opsim = pd.read_hdf("opsim.h5")
    
if initial_opsim is None:
    print("No visits found")
else:
    print(f"Found {len(initial_opsim)} visits")
    print(f"max day_obs in visits {initial_opsim.day_obs.max()}")
    print(f"running simulation for dayobs {day_obs}")

In [ ]:
# Configure scheduler and add initial visits

starting_scheduler, _, nside = simulate_lsst.setup_scheduler(config_script_path=config_script_path, 
                                                             config_ddf_script_path=config_ddf_script_path,
                                                             day_obs=day_obs, 
                                                             initial_opsim=initial_opsim.query("day_obs < @day_obs"))

In [ ]:
# Configure band scheduler
band_scheduler = simulate_lsst.setup_band_scheduler()

# Check what bands it expects for day_obs? 
conditions = Conditions(mjd=sunset.mjd)
print("Bands available", band_scheduler(conditions))

In [ ]:
# Could consider placing a single seeing value 
swap_seeing = False
if swap_seeing:
    # The fill value for missing DIMM values is 1
    seeing=1.0
else:
    seeing=None

    
starting_observatory, survey_info = simulate_lsst.setup_observatory(day_obs=day_downtime, nside=nside, 
                                                                    add_downtime=add_downtime, add_clouds=add_clouds, 
                                                                    seeing=seeing, real_downtime=real_downtime, 
                                                                    initial_opsim=initial_opsim)

In [ ]:
mask_days = 100
print(f"Modeled availability over next {mask_days} nights")
mask = np.where(survey_info['dayobsmjd'] < rn_dayobs.day_obs_to_time(day_obs).mjd + mask_days)
print(f"Total nighttime {survey_info["hours_in_night"][mask].sum()}, ")
print(f"total downtime {survey_info["downtime_per_night"][mask].sum()}, ")
print(f"available time {survey_info["hours_in_night"][mask].sum() - survey_info["downtime_per_night"][mask].sum()}")
print(f"Average availability over {mask_days} days {(survey_info["hours_in_night"][mask].sum() - survey_info["downtime_per_night"][mask].sum()) / survey_info["hours_in_night"][mask].sum()}")
print(f"Average availability over survey {survey_info['system_availability']}")

In [ ]:
print("Illustrate survey downtime assumptions")
for start, end in survey_info['downtimes']:
    x = np.floor(start - 0.5)
    plt.plot((x, x), (start - x, end - x) , color='k') 
x = np.floor(survey_info['sunsets12'] - 0.5)
y = survey_info['sunsets12'] - x
plt.fill_between(x, 0.8, y)
x = np.floor(survey_info['sunrises12'] - 0.5)
y = survey_info['sunrises12'] - x
plt.axvline(rn_dayobs.day_obs_to_time(day_obs).mjd, color='r', linestyle=':', linewidth=1.8)
plt.fill_between(x, y, 1.6)
plt.ylim(0.9, 1.5)
plt.xlim(survey_info['survey_start'].mjd, rn_dayobs.day_obs_to_time(day_obs).mjd + 200)
plt.xlabel("MJD", fontsize='large')
_ = plt.ylabel("Fraction of MJD", fontsize='large')
#plt.savefig("onsky_downtime.png", bbox_inches='tight')

In [ ]:
print("Visualize DDF accessibility on first night")
check_ddfs = True
if check_ddfs:
    ddfs = ddf_locations(skycoords=True)
    lsst_site = Site('LSST')
    times = np.arange(survey_info['survey_start'].mjd, survey_info['survey_end'].mjd, 1/24)
    times = np.arange(np.floor(Time.now().mjd - 0.5),survey_info['survey_end'].mjd, 0.25/24)
    times = np.arange(sunset.mjd - 0.1, sunrise.mjd + 0.1, 0.05/24)
    sunmoon = survey_info['almanac'].get_sun_moon_positions(times)
    moon_ra = sunmoon['moon_RA']
    moon_dec = sunmoon['moon_dec']
    sun_alt = np.degrees(sunmoon['sun_alt'])
    ddf_moon_dist = {}
    ddf_alt = {}
    for ddf in ddfs:
        ddf_moon_dist[ddf] = angular_separation(ddfs[ddf].ra.deg, ddfs[ddf].dec.deg, np.degrees(moon_ra), np.degrees(moon_dec))
        alt, az = approx_ra_dec2_alt_az(ddfs[ddf].ra.deg, ddfs[ddf].dec.deg, lsst_site.latitude, lsst_site.longitude, times, lmst=None)
        ddf_alt[ddf] = alt
    
    plt.figure()
    for ddf in ddfs:
        mask = np.where((sun_alt <= -12) & (ddf_alt[ddf] > 40))
        plt.plot(Time(times[mask], format='mjd').to_datetime(), ddf_moon_dist[ddf][mask], marker='.', linestyle='', label=ddf)
    plt.legend(loc=(1.01, 0.5))
    plt.axvline(sunset.to_datetime(), color='k', linestyle=':')
    plt.axvline(sunrise.to_datetime(), color='k', linestyle=':')
    plt.axhline(30)
    plt.xticks(rotation=90)
    plt.xlabel("MJD")
    plt.ylabel("Distance to moon (deg)")
    
    plt.figure()
    for ddf in ddfs:
        mask = np.where((sun_alt <= -12) & (ddf_alt[ddf] > 40))
        plt.plot(Time(times[mask], format='mjd').to_datetime(), ddf_alt[ddf][mask], marker='.', linestyle='', label=ddf)
    plt.legend(loc=(1.01, 0.5))
    plt.axvline(sunset.to_datetime(), color='k', linestyle=':')
    plt.axvline(sunrise.to_datetime(), color='k', linestyle=':')
    plt.axhline(30)
    plt.xticks(rotation=90)
    plt.xlabel("MJD")
    plt.ylabel("Altitude (deg)")

In [ ]:
reset = True
if reset:
    observatory = copy.deepcopy(starting_observatory)
    scheduler = copy.deepcopy(starting_scheduler)

In [ ]:
# Run the simulation for some nights
print("Running simulation")
observations, scheduler, observatory, rewards, obs_rewards, survey_info = simulate_lsst.run_sim(scheduler=scheduler, 
                                                                                                    band_scheduler=band_scheduler,
                                                                                                    observatory=observatory,
                                                                                                    survey_info=survey_info,
                                                                                                    day_obs=day_obs,
                                                                                                    sim_nights=sim_nights, 
                                                                                                    keep_rewards=False)

In [ ]:
# Need to start at a specific time inside the night?

# sunset, sunrise = rn_dayobs.day_obs_sunset_sunrise(day_obs, sun_alt=-12)
# start = sunset.mjd - 0.1 /24
# end = sunrise.mjd

# with warnings.catch_warnings():
#     warnings.simplefilter("ignore", RuntimeWarning)
#     vals = sim_runner(
#         observatory,
#         scheduler,
#         band_scheduler=fs,
#         sim_start_mjd=start,
#         sim_duration=end-start, 
#         record_rewards=False,
#         verbose=True,
#     )
# observatory = vals[0]
# scheduler = vals[1]
# observations = vals[2]
# if len(vals) == 5:
#     rewards = vals[3]
#     obs_rewards = vals[4]

In [ ]:
if sim_nights == 1:

    targets = pd.DataFrame(observations)
    
    from zoneinfo import ZoneInfo
    tz = ZoneInfo("Chile/Continental")
    tz_utc = ZoneInfo("UTC")
    telescope = "Simonyi"

    site = Site('LSST')
    almanac = Almanac()
    night_events = almanac.get_sunset_info(evening_date=rn_dayobs.day_obs_int_to_str(day_obs), longitude=site.longitude_rad)

    def mjd_to_datetime(mjd, scale='utc', timezone=tz):
        return Time(mjd, format='mjd', scale=scale).utc.to_datetime(timezone=timezone)
        
    eps = 1
    fig, ax = plt.subplots(figsize=(13, 8))
    ax_utc = ax.twiny()
    
    ax.set_title(f"{telescope} DAYOBS {day_obs}", pad=20)
    
    # Shade astronomical events
    ax.fill_between([mjd_to_datetime(night_events['sun_n12_setting']), 
                      mjd_to_datetime(night_events['sun_n18_setting'])],
                     2.5, 0.0, color='lightgray', alpha=0.3)
    ax.fill_between([mjd_to_datetime(night_events['sunset']), 
                     mjd_to_datetime(night_events['sun_n12_setting'])], 
                      2.5, 0.0, color='gray', alpha=0.3)
    ax.fill_between([mjd_to_datetime(night_events['sun_n18_rising']), 
                     mjd_to_datetime(night_events['sun_n12_rising'])],
                     2.5, 0.0, color='lightgray', alpha=0.3)
    ax.fill_between([mjd_to_datetime(night_events['sun_n12_rising']), 
                     mjd_to_datetime(night_events['sunrise'])],
                     2.5, 0.0, color='gray', alpha=0.3)
    # Azimuth constraint period
    ax.fill_between([mjd_to_datetime(night_events['sunrise'] - 3/24),
                    mjd_to_datetime(night_events['sun_n18_rising'])],
                    2.5, 0.0, color='pink', alpha=0.1)
    
    if not np.isnan(night_events['moonrise']):
        ax.axvline(mjd_to_datetime(night_events['moonrise']), linestyle='-', color='blue', alpha=0.3)
    if not np.isnan(night_events['moonset']):
        ax.axvline(mjd_to_datetime(night_events['moonset']), linestyle='-', color='red', alpha=0.3)
    
    colors = cc.glasbey_category10
    # Assign distinct target sets with different colors
    marker_colors =  {}
    marker_style = {}
    labels = {}
    count = 0
    if len(targets) > 0:
        for sp in targets.science_program.unique():
            if sp in programs:
                for obs in targets.observation_reason.unique():
                    marker_colors[obs] = colors[count]
                    marker_style[obs] = 'o'
                    labels[obs] = obs
                    count += 1
            else:
                for sp in targets.science_program.unique():
                    marker_colors[sp] = colors[count]
                    marker_style[sp] = '^'
                    labels[sp] = sp
                    count += 1
    
    if len(targets) > 0:
        visit_alpha = 0.7
        for program in programs:
            q = targets.query("science_program == @program")
            for sp in q.observation_reason.unique():
                qq = q.query("observation_reason == @sp")
                label = sp
                ax.plot(mjd_to_datetime(qq.mjd, 'tai'), qq.airmass, 
                        marker=marker_style[sp], linestyle='',
                        color=marker_colors[sp], label=label,
                        alpha=visit_alpha, markerfacecolor='none', zorder=3)
        # then the remainder
        q = targets.query("science_program not in @programs")
        for sp in q.science_program.unique():
            qq = q.query("science_program == @sp")
            label = sp
            ax.plot(mjd_to_datetime(qq.mjd, 'tai'), qq.airmass, 
                    marker=marker_style[sp], linestyle='',
                    color=marker_colors[sp], markersize=9, label=label,
                    alpha=visit_alpha+0.3, markerfacecolor='none', zorder=3)

    # Add actual visits
    if len(initial_opsim) > 0:
        q = initial_opsim.query("day_obs == @day_obs")
        for b in q.band.unique():
            qq = q.query("band == @b")
            ax.plot(mjd_to_datetime(qq.observationStartMJD), qq.airmass, label=f"{b} science",
                    alpha=0.6, marker='.', linestyle='', color=band_colors[b], zorder=0)
        if len(q) > 0:
            print(f"Consdb reported {len(q)} visits. Simulation generated {len(targets)} visits.")
        
    ax.legend(loc=(1.01, 0.0), ncol=2)
    
    x0 = night_events['sunset']+30/60/24
    
    ax.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24), 
             mjd_to_datetime(night_events['sunrise']-30/60/24))
    ax_utc.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24, 'utc', timezone=tz_utc), 
             mjd_to_datetime(night_events['sunrise']-30/60/24, 'utc', timezone=tz_utc))
    
    ax.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24), 
             mjd_to_datetime(night_events['sunrise']-30/60/24))
    ax_utc.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24, 'utc', timezone=tz_utc), 
             mjd_to_datetime(night_events['sunrise']-30/60/24, 'utc', timezone=tz_utc))
    
    # Set ticks relevant sides
    ax.tick_params(axis="x", bottom=True, top=False, labelbottom=True, labeltop=False)
    ax_utc.tick_params(axis="x", bottom=False, top=True, labelbottom=False, labeltop=True)
    
    # Rotate and align bottom ticklabels
    plt.setp([tick.label1 for tick in ax.xaxis.get_major_ticks()], rotation=45,
             ha="right", va="center", rotation_mode="anchor")
    
    # Rotate and align top ticklabels
    plt.setp([tick.label2 for tick in ax_utc.xaxis.get_major_ticks()], rotation=45,
             ha="left", va="center",rotation_mode="anchor")
    
    plt.grid(True, alpha=0.2)
    
    plt.ylim(2.5, 0.9)
    
    ax.set_ylabel("Airmass", fontsize="large")
    ax.set_xlabel(f"Time ({tz})", fontsize="large")
    ax_utc.set_xlabel("Time (UTC)", fontsize='large')
    _ = plt.ylabel("Airmass", fontsize="large")

In [ ]:
with_visits = True
only_visits = False
if day_obs < today_dayobs:
    print("Comparison of model vs. actual seeing - for past nights only")

    fig, ax = plt.subplots(1, 1, figsize=(12, 6))

    if not only_visits:
        dimm = endpoints['efd'].select_time_series("lsst.sal.DIMM.logevent_dimmMeasurement", ['fwhm'], sunset, sunrise, index=1)
        ax.plot(dimm.index, dimm.fwhm, linestyle='-', marker='.', linewidth=1, label="dimm1")
        dimm2 = endpoints['efd'].select_time_series("lsst.sal.DIMM.logevent_dimmMeasurement", ['fwhm'], sunset, sunrise, index=2)
        if len(dimm2) > 0:
            ax.plot(dimm2.index, dimm2.fwhm, linestyle='-', marker='.',  linewidth=1, label="dimm2")
        ringss = endpoints['efd'].select_time_series("lsst.sal.ESS.logevent_ringssMeasurement", ['fwhmFree', 'fwhmSector', 'fwhmScintillation'], sunset, sunrise)
        ax.plot(ringss.index, ringss.fwhmFree, alpha=0.5, linestyle='-', marker='.',  label="ringss")
        print(f"Dayobs {day_obs}")
        print(f"DIMM mean FWHM {dimm.fwhm.mean():.2f}")
        ax.plot(Time(observations['mjd'], format='mjd').to_datetime(), observations['FWHM_500'], linestyle=':', marker='.',  color='r', label="model fwhm500")

    if with_visits:
        ax.plot(Time(observations['mjd'], format='mjd').to_datetime(), observations['FWHMeff'], 'r*', markersize=9, linestyle='', alpha=0.6, label="Simulated visit fwhm")
        q = initial_opsim.query("day_obs == @day_obs")
        ax.plot(Time(q.observationStartMJD, format='mjd').to_datetime(), q.seeingFwhmEff, 'k*', markersize=9, linestyle='', alpha=0.6, label="Measured visit fwhm")
    
    ax.legend(loc=(1.01, 0.3))
    
    _ = ax.set_xlim(Time(targets.mjd - 0.5/24, format='mjd').to_datetime().min(), Time(targets.mjd + 0.5/24, format='mjd').to_datetime().max())
    _ = plt.xticks(rotation=90)

In [ ]:
q = initial_opsim.query("day_obs == @day_obs")
obs_visit_gap = (q.observationStartMJD[1:].values - q.obs_end_mjd[:-1].values) * 60 * 60 * 24
sim_visit_gap = (observations['mjd'][1:] - (observations['mjd'][:-1] + observations['visittime'][:-1]/60/60/24)) * 60 *60 *24
bins = np.arange(0, 30, 0.5)
_ = plt.hist(obs_visit_gap, bins=bins, alpha=0.6, label="observation")
_ = plt.hist(sim_visit_gap, bins=bins, alpha=0.6, label="sim")
plt.legend()
_ = plt.xlabel("Time between visits (seconds)")

In [ ]:
# Make map on-sky of sim visits
print("Map of visits from this (sim, maybe also real) night only")
if sim_nights == 1:
    snvis = {}
    m_nvis = maf.CountMetric(col='mjd', metric_name = "Nvisits")
    s = maf.HealpixSlicer(nside=64, lon_col='RA', lat_col='dec', rot_sky_pos_col_name = 'rotSkyPos', lat_lon_deg=False)
    for b in ['all']:
        constraint = f"{b}"
        if b == 'all':
            opsvis = pd.DataFrame(observations).to_records()
        else:
            opsvis = pd.DataFrame(observations).query("band == @b").to_records()
        snvis[b] = maf.MetricBundle(m_nvis, s, constraint)
        g = maf.MetricBundleGroup({f'single night nvisits {b}': snvis[b]}, None)
        if len(opsvis) > 0:
            g.run_current(constraint, opsvis)

    background = plot.get_background(nside=64)
    mval = snvis['all'].metric_values
    vmax = np.percentile(mval.compressed(), 95)
    alpha = np.where(np.isnan(background), 0, background)
    alpha = np.where(alpha> 0.5, 0.5, alpha)
    alpha = np.where(mval.filled(0) > 0, 1, alpha)
    plot.hp_moll(mval.filled(0), alpha=alpha, vmin=0, vmax=vmax, label='Simulated night')

# Make map on-sky of real visits
if sim_nights == 1 and day_obs < today_dayobs:
    vnvis = {}
    m_nvis = maf.CountMetric(col='observationStartMJD', metric_name = "Nvisits")
    s = maf.HealpixSlicer(nside=64, lon_col='fieldRA', lat_col='fieldDec', rot_sky_pos_col_name = 'rotSkyPos', lat_lon_deg=True)
    for b in ['all']:
        constraint = f"{b}"
        if b == 'all':
            opsvis = pd.DataFrame(initial_opsim.query("day_obs == @day_obs")).to_records()
        else:
            opsvis = pd.DataFrame(observations).query("day_obs == @day_obs and band == @b").to_records()
        vnvis[b] = maf.MetricBundle(m_nvis, s, constraint)
        g = maf.MetricBundleGroup({f'single night nvisits {b}': vnvis[b]}, None)
        if len(opsvis) > 0:
            g.run_current(constraint, opsvis)

    mval = vnvis['all'].metric_values
    vmax = np.percentile(mval.compressed(), 95)
    plot.hp_moll(mval.filled(0), alpha=alpha, vmin=0, vmax=vmax, label='Consdb night')

In [ ]:
filename = "test.db"
if sim_nights == 1:
    # Only new observations?
    obsdf = lsst_support.save_opsim(observatory, observations, initial_opsim=None, filename=filename)
else:
    # All observations?
    obsdf = lsst_support.save_opsim(observatory, observations, initial_opsim=initial_opsim, filename=filename)

def add_dayobs(x):
    mjdfloor = Time(np.floor(x.observationStartMJD - 0.5) + 0.5, format="mjd", scale="tai")
    return rn_dayobs.day_obs_str_to_int(mjdfloor.isot.split("T")[0])

obsdf["day_obs"] = obsdf.apply(add_dayobs, axis=1)

In [ ]:
# Check all visits have metadata

def split_regions(x):
    regions = set()
    for k in x: #.target_name:
        regions = regions.union(set([kk.replace(' ', '') for kk in k.split(',')]))
    regions = list(regions)
    regions.sort()
    return regions
print("Regions observed", split_regions(obsdf.target_name.unique()))
print("Science programs", obsdf.science_program.unique())
print("Observation Reasons / Surveys", np.sort(obsdf.observation_reason.unique()))

In [ ]:
# Visualize DDF season coverage
# q = pd.DataFrame(scheduler.survey_lists[2][0].obs_wanted).query("observation_reason.str.contains('cosmos') and mjd < (@SURVEY_START_MJD + 365*2)")
# _ = plt.hist(q.mjd, bins=365*2)
# dd = np.searchsorted(survey_info['sunrises12'], q.mjd)
# before_sunrise = (survey_info['sunrises12'][dd] - q.mjd.values) * 24
# mm = np.where((q.mjd.values < Time("2026-03-01T12:00:00").mjd) & (before_sunrise > 3))
# plt.plot(q.mjd.values[mm], before_sunrise[mm], 'r.')

In [ ]:
compare_baseline = False
if compare_baseline:
    # compare to baseline at same number of nights (increasingly bad comparison right now)
    opsdb = get_baseline()
    sim_name = opsdb.split("/")[-1].replace('.db', '')
    conn = sqlite3.connect(opsdb)
    b_visits = pd.read_sql(f"select * from observations where night < 365", conn)
    
    tconsdb = obsdf.observationStartMJD.max() - obsdf.observationStartMJD.min()
    print(f"comparing visits within {tconsdb} days of start")
    baseline_visits = b_visits.query("observationStartMJD - observationStartMJD.min() < @tconsdb")
    
    xx = baseline_visits.groupby("observation_reason").agg({'observationStartMJD': "count"}).rename({"observationStartMJD": sim_name}, axis=1)
    xa = obsdf.groupby("observation_reason").agg({'observationStartMJD': 'count'}).rename({"observationStartMJD": "sim config"}, axis=1)
    print(len(baseline_visits), len(obsdf), len(obsdf)/len(baseline_visits))
    x = pd.concat([xa, xx], axis=1)
    x.loc['total'] = x.sum(axis=0)
    display(x)

In [ ]:
ddf_visits = obsdf.query("observation_reason.str.contains('DD') or observation_reason.str.contains('ddf')").copy()
if len(ddf_visits) > 0:
    print("n visits per band")
    ddf_visits.loc[:, 'observation_reason'] = ddf_visits.observation_reason.str.lower()
    ss = ddf_visits.groupby(["observation_reason", "band"]).agg({'observationStartMJD': 'count'})
    ss = ss.reset_index('band').pivot(columns=["band"]).droplevel(0, axis=1)
    ss['all'] = ss.sum(axis=1)
    order = ['u', 'g', 'r', 'i', 'z', 'y', 'all']    
    display(ss.query("observation_reason.str.contains('dd')")[[o for o in order if o in ss.columns]])
    
    print("n days per band")
    ss = ddf_visits.groupby(["observation_reason", "band"]).agg({'day_obs': 'nunique'})
    ss = ss.reset_index('band').pivot(columns=["band"]).droplevel(0, axis=1)
    ss['all'] = ddf_visits.groupby("observation_reason").agg({'day_obs': 'nunique'})
    display(ss.query("observation_reason.str.contains('dd')")[[o for o in order if o in ss]])

In [ ]:
run_calc = True
if run_calc:
    nvisits = {}
    coadd = {}
    m_nvis = maf.CountMetric(col='observationStartMJD', metric_name = "Nvisits")
    m_coadd = maf.Coaddm5Metric(m5_col='fiveSigmaDepth')
    s = maf.HealpixSlicer(nside=64, lon_col='fieldRA', lat_col='fieldDec', rot_sky_pos_col_name = 'rotSkyPos')
    for b in ['u', 'g', 'r', 'i', 'z', 'y', 'all']:
        constraint = f"{b}"
        if b == 'all':
            opsvis = obsdf.to_records()
        else:
            opsvis = obsdf.query("band == @b").to_records()
        nvisits[b] = maf.MetricBundle(m_nvis, s, constraint)
        coadd[b] = maf.MetricBundle(m_coadd, s, constraint)
        g = maf.MetricBundleGroup({f'nvisits {b}': nvisits[b], f'coadd {b}': coadd[b]}, None)
        if len(opsvis) > 0:
            g.run_current(constraint, opsvis)            

In [ ]:
print("Plots of all visits + sim visits")
background = plot.get_background(nside=64)

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(16, 10),)
axdict = {"u": ax[0][0], "g": ax[0][1], "r": ax[0][2],
          "i": ax[1][0], "z": ax[1][1], "y": ax[1][2], "all": None}
for b in ["u", "g", "r", "i", "z", "y"]:
    if nvisits[b].metric_values is None:
        continue
    if len(nvisits[b].metric_values.compressed()) > 1:
        vmax = np.percentile(nvisits[b].metric_values.compressed(), 95)
    else:
        vmax = None
    label_dec = False
    if b == 'u' or b == 'i':
        label_dec = True
    fig = plot.make_plot(nvisits[b], background=background, proj='McBryde', vmax=vmax, ax=axdict[b], title=f"LSSTCam band {b}", label_dec=label_dec)
fig.tight_layout()
#fig.savefig(os.path.join(out_dir, f"lsstcam_nvisits_band.png"), bbox_inches='tight')

vmax = np.percentile(nvisits['all'].metric_values.compressed(), 95)
fig = plot.make_plot(nvisits['all'], background=background, proj='mcbryde', vmin=None, vmax=vmax, ax=None, title=f"LSSTCam visits")
#fig.savefig(os.path.join(out_dir, f"lsstcam_nvisits.png"), bbox_inches='tight')
fig = plot.make_plot(nvisits['all'], background=background, proj='laea', vmin=None, vmax=vmax, ax=None, title=f"LSSTCam visits")
#fig.savefig(os.path.join(out_dir, f"lsstcam_laea_nvisits.png"), bbox_inches='tight')